In [ ]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.7 MB/s eta 0:00:00


In [ ]:
import keras

#**Loading Dataset**

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images,train_labels), (test_images,test_labels) = fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


#**Bringing all images within the range of 0-255**

In [ ]:
train_images=train_images/255.0
test_images=test_images/255.0

#**Restricting images to 28X28 pixels**

In [ ]:
train_images = train_images.reshape(len(train_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)

#**Creating Model**

In [ ]:
def build_model(hyperparameter):
  filter_1 = keras.layers.Conv2D(
        filters = hyperparameter.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size = hyperparameter.Choice('conv_1_kernel', values = [3,5]),
        activation = 'relu',
        input_shape = (28,28,1)
    )

  filter_2 = keras.layers.Conv2D(
        filters = hyperparameter.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size = hyperparameter.Choice('conv_2_kernel', values = [3,5]),
        activation = 'relu'
    )

  hidden_1 = keras.layers.Dense(
        units = hyperparameter.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation = 'relu'
    )

  flatten = keras.layers.Flatten()

  # Creating a model just like we do in ANN
  model = keras.models.Sequential()

  # Aplying filters in the first step
  model.add(filter_1)

  # Applying filters again in the second step
  model.add(filter_2)

  # Flattening layer ==> Point where we begin with our NN training
  model.add(flatten)

  # Proceeding to our first hidden layer
  model.add(hidden_1)

  # The final/output layer should have 10 neurons since there are 10 output classes in the dataset ==> 1.T-shirt/top 2.Trouser 3.Pullover 4.Dress 5.Coat 6.Sandal 7.Shirt 8.Sneaker 9.Bag 10.Ankle boot
  model.add(keras.layers.Dense(10, activation='softmax'))

  # Finally setting up the loss function and optimisers
  model.compile(optimizer = keras.optimizers.Adam(hyperparameter.Choice('learning_rate', values=[0.01, 0.001])),
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

  return model

#**Finding the best hyperparameters for tuning -- Keras Tuner**

In [ ]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search = RandomSearch(build_model,
                          objective = 'val_accuracy',
                          max_trials = 5,
                          directory = 'output',
                          project_name = "Mnist Fashion")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
tuner_search.search(train_images,
                    train_labels,
                    epochs = 3,
                    validation_split = 0.1)

Trial 5 Complete [00h 39m 31s]
val_accuracy: 0.8696666955947876

Best val_accuracy So Far: 0.8696666955947876
Total elapsed time: 01h 59m 14s


In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 26, 26, 128)         │           1,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 22, 22, 48)          │         153,648 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 23232)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 48)                  │       1,115,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │             490 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,270,602 (4.85 MB)

 Trainable params: 1,270,602 (4.85 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 785s 464ms/step - accuracy: 0.8732 - loss: 0.3392 - val_accuracy: 0.8690 - val_loss: 0.3586
Epoch 5/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 791s 458ms/step - accuracy: 0.8793 - loss: 0.3218 - val_accuracy: 0.8747 - val_loss: 0.3477
Epoch 6/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 801s 457ms/step - accuracy: 0.8856 - loss: 0.3029 - val_accuracy: 0.8720 - val_loss: 0.3710
Epoch 7/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 821s 469ms/step - accuracy: 0.8908 - loss: 0.2897 - val_accuracy: 0.8575 - val_loss: 0.4057
Epoch 8/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 808s 472ms/step - accuracy: 0.8914 - loss: 0.2883 - val_accuracy: 0.8595 - val_loss: 0.3971
Epoch 9/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 766s 454ms/step - accuracy: 0.8974 - loss: 0.2771 - val_accuracy: 0.8540 - val_loss: 0.4362
Epoch 10/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 801s 453ms/step - accuracy: 0.9046 - loss: 0.2609 - val_accuracy: 0.8627 - val_loss: 0.4348
